<a href="https://colab.research.google.com/github/tduenas1/cherryblossoms/blob/main/cb_scrape_weather_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
from google.colab import files
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import time


Scrape function for the cherry blossom data using beatiful soup

In [ ]:
def scrape_cb(section, division, start_year, end_year):
    base_url = "https://www.cballtimeresults.org/performances?division=Overall+Men&section=10M&sex=M&utf8=%E2%9C%93"
    scrape_data = []
    for year in range(start_year, end_year + 1):
        page = 1
        while True:
            #from the base url, this parameters are modified in each loop or function call
            url = f"{base_url}&section={section}&division={division}&year={year}&page={page}"

            try:
                # request data from url
                response = requests.get(url)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, 'html.parser')

                # extract all table rows
                table = soup.find('table')
                rows = table.find_all('tr')

                # break if there is no data to add (checking for 404 did not work)
                if len(rows) <= 1:
                    break

                for row in rows:
                    # Extract data from each row and process it as needed
                    cells = row.find_all('td')
                    row_data = [cell.get_text().replace('\n', '') for cell in cells]
                    # Prevent adding empty rows
                    if any(cell for cell in row_data[1:]):
                        scrape_data.append(row_data)

            except requests.exceptions.RequestException as e:
                print(f"Error: {e}")
                break

            # Move to the next page
            page += 1
        # log scraping status
        print(section, " ", division, " ", year, " scrape complete")

        df_temp = pd.DataFrame(scrape_data)

    return df_temp



# Do Not Run
main calling of scrape functions for each division and race length
(this took 97 minutes to run)

In [ ]:

# track scrape time
start_time = time.time()

section = "10M"
division = "Overall+Men"
start_year = 1973
end_year = 2020

df1 = scrape_cb(section, division, start_year, end_year)

# 5k race started 2006
section = "5K"
division = "Overall+Men"
start_year = 2006
end_year = 2020

df2 = scrape_cb(section, division, start_year, end_year)

section = "10M"
division = "Overall+Women"
start_year = 1973
end_year = 2020

df3 = scrape_cb(section, division, start_year, end_year)

section = "5K"
division = "Overall+Women"
start_year = 2006
end_year = 2020

df4 = scrape_cb(section, division, start_year, end_year)

# evaluate scrape time
end_time = time.time()
run_time = end_time - start_time
print("runtime:", run_time, "s", " ", run_time / 60, "m")




combine all dfs into one file

In [ ]:
# join df's together vertically
cb_master_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

new_column_headers = ['Year', 'Name', 'Age', 'Time', 'Pace', 'PiS/TiS', 'Division', 'PiD/TiD', 'Hometown']
cb_master_df.columns = new_column_headers

split the PiS/TiS and PiD/Tid cols. Those columns are not useful as a fraction, but we might want to know how many participants were in each division or sex and what position they came in.

In [ ]:
# splitting columns
cb_master_df[['Year','Distance']] = cb_master_df['Year'].str.split(' ', expand=True)
cb_master_df[['pos_by_sex','total_by_sex']] = cb_master_df['PiS/TiS'].str.split('/', expand=True)
cb_master_df[['pos_by_division','total_by_division']] = cb_master_df['PiD/TiD'].str.split('/', expand=True)

# extract sex from (M) into new column with one character (i have absolutely no idea why it looks like that)
cb_master_df['Sex'] = cb_master_df['Name'].str.extract(r'\(([^)]+)\)')

#remove 4 string characters for (M) in name
cb_master_df['Name'] = cb_master_df['Name'].str[:-4]

# delete unnecessary cols
cb_master_df = cb_master_df.drop('PiS/TiS', axis=1)
cb_master_df = cb_master_df.drop('PiD/TiD', axis=1)



df to csv and download file

In [ ]:
cb_master_df.to_csv("cb_master.csv", index=False)
files.download("cb_master.csv")

# Run code blocks starting here

I am reuploading the csv file here so that I don't have to run the scraping calls every time I want to use the data

In [ ]:
# upload CSV from local hard drive (original sheet that I shared)
upload = files.upload()
df = pd.read_csv(io.StringIO(list(upload.values())[0].decode('utf-8')), low_memory=False)

Saving cb_master.csv to cb_master.csv


In [ ]:
# split by distance to join different weather dates
df_10M = df[df['Distance'] == '10M']
df_5K = df[df['Distance'] == '5K']

In [ ]:
result = df_5K[(df_5K['Sex'] == 'M') & (df_5K['Year'] == 2013)]

print(result.head(5))

        Year             Name Age     Time  Pace Division         Hometown  \
181605  2013     Nathan Davis  12  0:17:22  5:36    M0119    Frederick, MD   
181606  2013   Thomas Johnson  27  0:17:27  5:38    M2529    Arlington, VA   
181607  2013  Ariel Laguilles  34  0:17:55  5:47    M3034   Washington, DC   
181608  2013    Nahom Teshome  15  0:18:11  5:52    M0119  Springfield, VA   
181609  2013   Paul Vasilescu  29  0:18:17  5:54    M2529   Washington, DC   

       Distance pos_by_sex  total_by_sex pos_by_division  total_by_division  \
181605       5K          1         547.0               1               35.0   
181606       5K          2         547.0               1               87.0   
181607       5K          3         547.0               1              110.0   
181608       5K          4         547.0               2               35.0   
181609       5K          5         547.0               2               87.0   

       Sex  
181605   M  
181606   M  
181607   M  
1816

The hometown data has many inconsistances. Stephen Sambu has won the event several times in recent years and lists "   , Kenya" , "Oakland, CA" and "Oakland, Kenya" as various entries for that column. It appears that many international competitors have inconsitent city and countries listed. As is consistent with top olympic distance runners many of the top competitors are from Kenya, Uganda, and Ethiopa.

# Weather data

These are the race dates for every year from the "Rite of Spring" pdf. Verified all dates for the 10M. The 5K is assumed to always be held the day before.

In [ ]:
race_date = ['1973-04-01', '1974-03-31', '1975-04-06', '1976-04-04',
             '1977-04-03', '1978-04-02', '1979-04-01', '1980-03-30',
             '1981-04-05', '1982-04-04', '1983-03-27', '1984-04-01',
             '1985-03-31', '1986-04-06', '1987-04-05', '1988-03-27',
             '1989-04-02', '1990-04-01', '1991-03-31', '1992-04-05',
             '1993-04-04', '1994-04-10', '1995-04-09', '1996-03-31',
             '1997-04-06', '1998-04-05', '1999-04-11', '2000-04-09',
             '2001-04-08', '2002-04-07', '2003-04-06', '2004-04-04',
             '2005-04-03', '2006-04-02', '2007-04-01', '2008-04-06',
             '2009-04-05', '2010-04-11', '2011-04-03', '2012-04-01',
             '2013-04-07', '2014-04-06', '2015-04-12', '2016-04-03',
             '2017-04-02', '2018-04-07', '2019-04-07']

race_date_5K = ['2006-04-01', '2007-03-31', '2008-04-05',
             '2009-04-04', '2010-04-10', '2011-04-02', '2012-03-31',
             '2013-04-06', '2014-04-05', '2015-04-11', '2016-04-02',
             '2017-04-01', '2018-04-06', '2019-04-06']



In order to obtain weather data I sumbitted a data request from the National Oceanic and Atmospheric Administration (NOAA) and the National Centers for Environmental Information. The closest option to the marathon was the Washington Reagon Airport, Arlington, VA which is physically closer to the marathon location than the next available option which was the National Arboretum, DC. The Reagon airport is also situatuated on the banks of the Potomac across from the National Mall where most of the race takes place.

In [ ]:
# Upload a CSV file from your local hard drive
upload = files.upload()

# Read the uploaded CSV file into a DataFrame
weather = pd.read_csv(io.StringIO(list(upload.values())[0].decode('utf-8')))

Saving NOAA_weather_1973_2020.csv to NOAA_weather_1973_2020.csv


In [ ]:
weather = weather[['DATE', 'PRCP', 'TMAX', 'TMIN']]

weather.head(5)

,DATE,PRCP,TMAX,TMIN
0,1973-01-01,0.00,64,46
1,1973-01-02,0.00,48,32
2,1973-01-03,0.18,40,29
3,1973-01-04,0.29,49,35
4,1973-01-05,0.00,47,38


Now we want to exctract only the first sunday in april from every year

(must run following blocks in order as event_weather gets overwritten)

**Weather Codes present in data:**

WT01 - Fog, ice fog, or freezing fog (may include heavy fog)

WT02 - Heavy fog or heaving freezing fog (not always distinguished from fog)

WT03 - Thunder

WT05 - Hail (may include small hail)

WT08 - Smoke or haze

WT13 - Mist

WT14 - Drizzle

WT16 - Rain (may include freezing rain, drizzle, and freezing drizzle)"

WT18 - Snow, snow pellets, snow grains, or ice crystals

**Weather codes Not in data:**

WT04 - Ice pellets, sleet, snow pellets, or small hail"

WT06 - Glaze or rime

WT07 - Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction

WT09 - Blowing or drifting snow

WT10 - Tornado, waterspout, or funnel cloud"

WT11 - High or damaging winds

WT12 - Blowing spray

WT15 - Freezing drizzle

WT17 - Freezing rain

WT19 - Unknown source of precipitation

WV20 - Rain or snow shower

WT21 - Ground fog

WT22 - Ice fog or freezing fog

In [ ]:
# convert race_date to a pandas Series for comparison
race_date_series = pd.Series(race_date)
race_date_series_5k = pd.Series(race_date_5K)

# create a new DataFrame with rows from weather that match values in race_date
event_weather = weather[weather['DATE'].isin(race_date_series)]
event_weather_5K = weather[weather['DATE'].isin(race_date_series_5k)]

print(event_weather.head(2))

           DATE  PRCP  TMAX  TMIN
90   1973-04-01  0.50    67    55
454  1974-03-31  0.01    53    40


It appears that most of the coded weather events present on event days like fog and mist, would not impact running times. Other weather conditions such as rain would be better explained by the quantatative value of precipitation. There are no snow events and only two hail or freezing related codes present. In 2008 "Hail (may include small hail)" was recorded along with rain and high precipitation. However the low temperature recorded was 46 degrees. In 1987 "Snow, snow pellets, snow grains, or ice crystals" was recorded, but there was no recorded snowfall and the low temperature was 35, so it appears to be minor incident of ice crystals on the ground. For our further analysis we will discard the weather codes and rely solely on temperature and precipitation the primary weather factors impacting running performance.

TAVG (average temp was also dropped due to to many missing values.

(inches and fahrenheit requested)
from the NOAA documentation:

PRCP = Precipitation (mm or inches as per user preference, inches to hundredths on Daily Form pdf file)
TMAX = Maximum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on
Daily Form pdf file
TMIN = Minimum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on
Daily Form pdf file

In [ ]:
event_weather_join = event_weather.copy()
#create date column that matches for the join key
event_weather_join['Year'] = event_weather_join['DATE'].str[:4]
event_weather_join['Year'] = event_weather_join['Year'].astype(int)

event_weather_5K_join = event_weather_5K.copy()
event_weather_5K_join['Year'] = event_weather_5K_join['DATE'].str[:4]
event_weather_5K_join['Year'] = event_weather_5K_join['Year'].astype(int)


outer join by year to link all weather data to the cherry blossom data

(this overwrites the df so running it multiple times infinetly accumulates new cols)

In [ ]:
df_10M_weather = pd.merge(df_10M, event_weather_join, on='Year', how='outer')

df_5K_weather = pd.merge(df_5K, event_weather_5K_join, on='Year', how='outer')

In [ ]:
df = pd.concat([df_10M_weather, df_5K_weather], ignore_index=True)

In [ ]:
df.to_csv("cb_weather2.csv", index=False)
files.download("cb_weather2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>